In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import inspect
currentdir = os.path.dirname(os.path.abspath(
    inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(1, parentdir+'/src')
import torch
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
import time

In [ ]:
import gym
import stage.envs
from stage.tasks.kuka import KukaPETS
from stage.utils.nn import use_gpu

In [ ]:
savepath = parentdir + '/data/kuka/'

In [ ]:
data_train = np.load(savepath+'data_pd.npy')
kuka = KukaPETS(render=True, 
                action_parameterization='pd')
kuka.visualize_training_data(data_train, it_begin=94)
q = kuka.env.step_cost.desired.detach().cpu().numpy()
v = np.zeros(7)
kuka.env.set_state(q, v)

In [ ]:
kuka.env.close()